# Import models

In [ ]:
import tvm 
from tvm import relay
from tvm.contrib import relay_viz, graph_executor
import tvm.relay.testing.tf as tf_testing  

import tensorflow as tf 
try:
    tf_compat_v1 = tf.compat.v1
except ImportError:
    tf_compat_v1 = tf

import onnx

# import from tensorflow
with tf_compat_v1.gfile.GFile("tf_model_path", "rb") as f:
    graph_def = tf_compat_v1.GraphDef()
    graph_def.ParseFromString(f.read())
    graph = tf.import_graph_def(graph_def, name="")
    
    graph_def = tf_testing.ProcessGraphDefParam(graph_def) 

mod, params = relay.frontend.from_tensorflow(graph_def, layout=None) 

# import from onnx 
onnx_mod = onnx.load("onnx_model_path")
mod, params = relay.frontend.from_onnx(onnx_mod)
 
# build and save to lib
with tvm.transform.PassContext(opt_level = 1):
    target = tvm.target.Target("llvm", host="llvm")
    lib = relay.build(mod, target, params = params)
    lib.export_library("lib_path")

# load from a lib
lib = tvm.runtime.load_module("lib_path")

# run inference
dev = tvm.cpu(0)
m = graph_executor.GraphModule(lib["default"](dev)) 
m.set_input("name", tvm.nd.array([]))  
m.run()  
tvm_output = m.get_output(0)

# visualize the relay tree 
viz = relay_viz.RelayVisualizer(mod)
viz.render()

# Relay

In [ ]:
import tvm
from tvm import relay 
from tvm.contrib import graph_executor

import numpy as np


def _get_func(shape, dtype):
    a = relay.var("a", shape=shape, dtype=dtype)
    b = relay.var("b", shape=shape, dtype=dtype) 

    zero = relay.const(0, "int32")
    one = relay.const(1, "float32")

    res = relay.qnn.op.add(a, b, one, zero, one, zero, one, zero)

    return relay.Function([a, b], res)


def _compile(func):
    mod = tvm.IRModule()
    mod["main"] = func

    with tvm.transform.PassContext(3): 
        lib = relay.build(mod, "llvm", params={}) 
    return lib

def _run(lib, a, b): 
    m = graph_executor.GraphModule(lib["default"](tvm.cpu(0)))

    m.set_input("a", tvm.nd.array(a))
    m.set_input("b", tvm.nd.array(b)) 

    m.run() 

    output = m.get_output(0)

    return output


a = np.random.randint(-128, 127, (1, 16), dtype="int16")
b = np.random.randint(-128, 127, (1, 16), dtype="int16") 

func = _get_func((1, 16), "int16")

lib = _compile(func)

out = _run(lib, a, b)

np.testing.assert_equal(a+b, out.numpy())





## Pattern Match

In [ ]:
import tvm
import tvm.relay as relay
from tvm.relay.dataflow_pattern import wildcard, is_op, rewrite, DFPatternCallback, FunctionPattern

class TestCallback(DFPatternCallback):
    def __init__(self):
        super(TestCallback, self).__init__()
        self.x = wildcard()
        self.y = wildcard()

        pattern = is_op('add')(self.x, self.y)
        pattern = FunctionPattern([wildcard(), wildcard()], pattern)(wildcard(), wildcard())
        self.pattern = pattern

    def callback(self, pre, post, node_map):
        print('here')
        x = node_map[self.x][0]
        y = node_map[self.y][0]
        return x - y

class TestCallback2(DFPatternCallback):
    def __init__(self):
        super(TestCallback2, self).__init__()
        self.x = wildcard()
        self.y = wildcard()

        pattern = is_op('add')(self.x, self.y)
        # pattern = FunctionPattern([wildcard(), wildcard()], pattern)(wildcard(), wildcard())
        self.pattern = pattern

    def callback(self, pre, post, node_map):
        print('here')
        x = node_map[self.x][0]
        y = node_map[self.y][0]
        return x - y
x = relay.var('x')
y = relay.var('y')
z = relay.var('z')
expr = (x + y) * z

p = wildcard() + wildcard()
fp = FunctionPattern([wildcard(), wildcard()], p)

print("EXPR: =====\n",expr)

expr_p = p.partition(expr)
print("EXPR_p: =====\n",expr_p)

expr_r = rewrite(TestCallback(), expr_p)
print("EXPR_r: =====\n",expr_r)

ee = rewrite(TestCallback2(), expr)
print("EXPR_r2:=====\n",ee)

## Traverse Relay

In [ ]:
def traverse_relay(ir_mod):
    global_vars = ir_mod.get_global_vars()
    graph_names = [] 
    for gv_node in global_vars:
        if gv_node.name_hint == "main":
            graph_names.insert(0, gv_node.name_hint)
        else:
            graph_names.append(gv_node.name_hint)

    node_to_id = {} 
    node_count_offset = 0 

    def traverse_expr(node): 
        optype = "" 
        
        if isinstance(node,relay.expr.Call): 
            args = [ arg.handle.value for arg in node.args] 
            if isinstance(node.op,tvm.ir.op.Op):
                optype = node.op.name
                name = optype + "_" + str(node.handle.value)[-4:]
            elif isinstance(node.op,tvm.ir.expr.GlobalVar):  
                name = node.op.name_hint.replace("tvmgen_default_versal_aie_main", "AIE")
                optype = "GVar"
            elif isinstance(node.op,relay.function.Function): 
                optype = "subgraph"
                name = "Func_" + str(node.handle.value)[-4:] 
            else:
                print("xxxxx",node)
            shape = node.checked_type if hasattr(node, "checked_type") else "" 
        elif isinstance(node,relay.expr.Var):
            if g != "main":
                return
            shape = node.checked_type if hasattr(node, "checked_type") else "" 
            node_to_id[node.handle.value] = {
                "args":[], "name":node.name_hint,"shape":shape,"optype":"Placeholder"
            } 
        elif isinstance(node,relay.expr.Tuple): 
            # shape = node.checked_type if hasattr(node, "checked_type") else "" 
            args = [ f.handle.value for f in node.fields]
            name = "tuple_" + str(node.handle.value)[-4:]
            shape = ""
            optype = "Concat"
        elif isinstance(node,relay.function.Function):
            print(node.astext())   
    

    for name in graph_names:
        node_count_offset += len(node_to_id)
        # node_to_id.clear()  
        relay.analysis.post_order_visit(ir_mod[name], traverse_expr)

 

# TE

In [ ]:

import tvm
from tvm import te 
from tvm.driver.build_module import get_binds

def sch2stmt(sch):
    sch = sch.normalize()
    infer_bounds = tvm.te.schedule.InferBound(sch) 
    stmt = tvm.te.schedule.ScheduleOps(sch, infer_bounds, False)
 

    return stmt

def stmt2primfunc(stmt, args):
    compact = tvm.te.schedule.VerifyCompactBuffer(stmt)
    binds, arg_list = get_binds(args, compact, None)
    func = tvm.te.schedule.SchedulePostProcToPrimFunc(arg_list, stmt, binds)
    return func


def test():
    m = te.size_var("m")
    n = te.size_var("n")
    m = te.const(32)
    n = te.const(64)
    A = te.placeholder((m, n), name="A")
    T = te.compute((m, n), lambda i, j: A[i, j])

    s = te.create_schedule(T.op)
    xo, yo, xi, yi = s[T].tile(T.op.axis[0], T.op.axis[1], x_factor=10, y_factor=5)

    stmt = sch2stmt(s)
    func = stmt2primfunc(stmt, [A])
    print(func)

    print('='*20)

if __name__ == "__main__":
    test()

## schedule

In [ ]:
# tile


# fuse

# split

# reorder

# vectorize

# ...

# Pass


## debug

In [ ]:
@tvm.instrument.pass_instrument
class PrintIR:
    """Print the name of the pass, the IR"""

    def run_before_pass(self, mod, info):
        pass
        # print("Running pass: ", info.name)
        # if info.name == "LowerTensorExpr":
        #     print("Final Relay IR:")
        #     print(mod.astext(show_meta_data=False))

    def run_after_pass(self, mod, info):
        print("=" * 20, info.name)
        if "SomeText" in info.name:
            print("=" * 20, info.name)
            print(mod.astext(show_meta_data=False))


with tvm.transform.PassContext(opt_level=3,config={},instruments=[PrintIR()],): 
    lib = relay.build(mod, "llvm", params={}) 
